In [1]:
# load libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold, GridSearchCV
from scipy.stats import spearmanr
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import shap

import warnings
import os

# suppress all warnings
warnings.filterwarnings('ignore')

# set seed
np.random.seed(42)

# set colours
GradientBlueRed_res = plt.cm.viridis(np.linspace(0, 0.8, 256))
newcmp = ListedColormap(GradientBlueRed_res, name='BlueRed')

In [2]:
#import pkg_resources

#installed_packages = pkg_resources.working_set
#for dist in installed_packages:
#    print(dist.project_name, dist.version)

Define functions for running models

In [3]:
def spearman_scorer(y_true, y_pred):
  """
  Helper function to create scorer
  """
  return spearmanr(y_true, y_pred).correlation


spearman_score = make_scorer(spearman_scorer, greater_is_better=True)

In [4]:
def runEN(X, y, path):
  """
  Run Elastic Net with 5foldCV
  Final model, selected with most common hyperparameters, fitted to full dataset
  SHAP evaluated on final model
  """

  # create dataframe to store results
  model_df = pd.DataFrame(columns=['Model', 'Fold', 'Spearman', 'Pearson', 'alpha', 'max_iter'])

  # initialize folds (5 folds, 80% train, 20% test)
  outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
  inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)

  # initialize variables to store results
  results = []
  hyperparams_list = []
  fold = 1

  # loop through each of the outer five folds
  for train_index, test_index in outer_cv.split(X):

    # split train and test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # initialize ElasticNet model
    en = ElasticNet()

    # specify parameters for optimization
    parameters = {
      'alpha': [0.1, 1, 10, 100],
      'l1_ratio': [0.2, 0.5, 0.8],
      'max_iter': [1000, 5000, 7500]
    }

    # identify optimal parameters
    reg = GridSearchCV(
        estimator = en,
        param_grid = parameters,
        cv=inner_cv,
        scoring=spearman_score,
        n_jobs=-1
    )

    # fit model
    reg.fit(X_train, y_train)

    # get best model & parameters
    reg_best = reg.best_estimator_

    best_params = reg.best_params_
    hyperparams_list.append(best_params)

    # get predicted values for test data
    y_pred = reg_best.predict(X_test)

    # compute correlations
    s_corr = spearmanr(y_test, y_pred).correlation

    # save model correlation and features
    results.append({
        "fold": fold,
        "spearman": s_corr,
        "alpha": best_params["alpha"],
        "l1_ratio": best_params["l1_ratio"],
        "max_iter": best_params["max_iter"]
    })

    fold += 1

  # save results to dataframe
  results_df = pd.DataFrame(results)
  results_df.to_csv((path+"/en.csv"), index=False)

  # get most common hyperparameters
  params_df = pd.DataFrame(hyperparams_list)
  final_params = params_df.mode().iloc[0].to_dict()

  print("\nSelected hyperparameters:", final_params)

  # fit final model
  final_model = ElasticNet(
      alpha=final_params["alpha"],
      l1_ratio=final_params["l1_ratio"],
      max_iter=int(final_params["max_iter"])
  )
  final_model.fit(X, y)

  # run shap
  explainer = shap.Explainer(final_model, X)
  shap_values = explainer(X)

  # summary plot
  plt.figure()
  shap.summary_plot(shap_values, X, show=False)
  for fc in plt.gcf().get_children():
      for fcc in fc.get_children():
          if hasattr(fcc, "set_cmap"):
              fcc.set_cmap(newcmp)
  plt.savefig((path+"/shap_summary.png"), dpi=300, bbox_inches='tight')
  plt.close()

  # heatmap
  plt.figure()
  shap.plots.heatmap(
      shap_values,
      instance_order=shap_values.sum(1),
      show=False
  )
  plt.savefig((path+"/heatmap.png"), dpi=300, bbox_inches='tight')
  plt.close()

In [5]:
# run models for each dataset pair
def runModels(df, pair, outdir):

  # specify column names
  var = {"gcsi_ccle": ["gdsc_median", "gcsi_ccle_spearman"],
        "gcsi_gdsc": ["ccle_median", "gcsi_gdsc_spearman"],
        "gdsc_ccle": ["gcsi_median", "gdsc_ccle_spearman"]}

  # get needed columns and remove NA
  #chr_cols = [f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY", "chrM"]
  #available_cols = [c for c in [var[pair][0], var[pair][1], "gc", "n_exon", "length", *chr_cols] if c in df.columns]
  #subset = df[available_cols].dropna()
  subset = df[[var[pair][0], var[pair][1], "gc", "n_exon", "length"]].dropna()

  # get X and y
  X = subset.drop(columns=[var[pair][1]]).rename(
    columns={
        var[pair][0]: "Median exp",
        "gc": "GC%",
        "n_exon": "No. Exons",
        "length": "Transcript Length",
    }
  )
  y = subset[[var[pair][1]]]

  # create folder to store results
  path = os.path.join(outdir, pair)
  os.makedirs(path, exist_ok=True)

  # run models
  print("--- running EN for " + pair)
  runEN(X, y, path)


In [6]:
# function to run models for each dataset
def runAllPairs(df, outdir):

  # load in dataset
  df = pd.read_csv(df)

  # run models
  print("\nStarting gCSI/CCLE:")
  runModels(df, "gcsi_ccle", outdir)

  print("\nStarting gCSI/GDSC:")
  runModels(df, "gcsi_gdsc", outdir)

  print("\nStarting GDSC/CCLE:")
  runModels(df, "gdsc_ccle", outdir)

Run models for each pipeline

In [7]:
runAllPairs("gene_stability.csv", "Gene_Expression")


Starting gCSI/CCLE:
--- running EN for gcsi_ccle

Selected hyperparameters: {'alpha': 1.0, 'l1_ratio': 0.8, 'max_iter': 1000.0}

Starting gCSI/GDSC:
--- running EN for gcsi_gdsc

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.5, 'max_iter': 1000.0}

Starting GDSC/CCLE:
--- running EN for gdsc_ccle

Selected hyperparameters: {'alpha': 1.0, 'l1_ratio': 0.2, 'max_iter': 1000.0}


In [8]:
runAllPairs("transcript_stability.csv", "Isoform_Expression")


Starting gCSI/CCLE:
--- running EN for gcsi_ccle

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.5, 'max_iter': 1000.0}

Starting gCSI/GDSC:
--- running EN for gcsi_gdsc

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.5, 'max_iter': 1000.0}

Starting GDSC/CCLE:
--- running EN for gdsc_ccle

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 1000.0}


In [9]:

runAllPairs("ciri_stability.csv", "CIRI2")


Starting gCSI/CCLE:
--- running EN for gcsi_ccle

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.5, 'max_iter': 1000.0}

Starting gCSI/GDSC:
--- running EN for gcsi_gdsc

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 1000.0}

Starting GDSC/CCLE:
--- running EN for gdsc_ccle

Selected hyperparameters: {'alpha': 1.0, 'l1_ratio': 0.5, 'max_iter': 1000.0}


In [10]:
runAllPairs("circ_stability.csv", "CIRCexplorer2")


Starting gCSI/CCLE:
--- running EN for gcsi_ccle

Selected hyperparameters: {'alpha': 1.0, 'l1_ratio': 0.2, 'max_iter': 1000.0}

Starting gCSI/GDSC:
--- running EN for gcsi_gdsc

Selected hyperparameters: {'alpha': 1.0, 'l1_ratio': 0.5, 'max_iter': 1000.0}

Starting GDSC/CCLE:
--- running EN for gdsc_ccle

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 1000.0}


In [11]:
runAllPairs("cfnd_stability.csv", "circRNA_finder")


Starting gCSI/CCLE:
--- running EN for gcsi_ccle

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 1000.0}

Starting gCSI/GDSC:
--- running EN for gcsi_gdsc

Selected hyperparameters: {'alpha': 1.0, 'l1_ratio': 0.5, 'max_iter': 1000.0}

Starting GDSC/CCLE:
--- running EN for gdsc_ccle

Selected hyperparameters: {'alpha': 0.1, 'l1_ratio': 0.5, 'max_iter': 1000.0}


In [12]:
runAllPairs("fcrc_stability.csv", "find_circ")


Starting gCSI/CCLE:
--- running EN for gcsi_ccle

Selected hyperparameters: {'alpha': 100.0, 'l1_ratio': 0.8, 'max_iter': 1000.0}

Starting gCSI/GDSC:
--- running EN for gcsi_gdsc

Selected hyperparameters: {'alpha': 100.0, 'l1_ratio': 0.5, 'max_iter': 1000.0}

Starting GDSC/CCLE:
--- running EN for gdsc_ccle

Selected hyperparameters: {'alpha': 100.0, 'l1_ratio': 0.5, 'max_iter': 1000.0}


In [13]:
!zip -r Gene_Expression.zip "Gene_Expression"
!zip -r Isoform_Expression.zip "Isoform_Expression"
!zip -r CIRI2.zip "CIRI2"
!zip -r CIRCexplorer2.zip "CIRCexplorer2"
!zip -r circRNA_finder.zip "circRNA_finder"
!zip -r find_circ.zip "find_circ"

  adding: Gene_Expression/ (stored 0%)
  adding: Gene_Expression/gcsi_ccle/ (stored 0%)
  adding: Gene_Expression/gcsi_ccle/en.csv (deflated 37%)
  adding: Gene_Expression/gcsi_ccle/shap_summary.png (deflated 13%)
  adding: Gene_Expression/gcsi_ccle/heatmap.png (deflated 20%)
  adding: Gene_Expression/gcsi_gdsc/ (stored 0%)
  adding: Gene_Expression/gcsi_gdsc/en.csv (deflated 41%)
  adding: Gene_Expression/gcsi_gdsc/shap_summary.png (deflated 12%)
  adding: Gene_Expression/gcsi_gdsc/heatmap.png (deflated 19%)
  adding: Gene_Expression/gdsc_ccle/ (stored 0%)
  adding: Gene_Expression/gdsc_ccle/en.csv (deflated 38%)
  adding: Gene_Expression/gdsc_ccle/shap_summary.png (deflated 12%)
  adding: Gene_Expression/gdsc_ccle/heatmap.png (deflated 18%)
  adding: Isoform_Expression/ (stored 0%)
  adding: Isoform_Expression/gcsi_ccle/ (stored 0%)
  adding: Isoform_Expression/gcsi_ccle/en.csv (deflated 43%)
  adding: Isoform_Expression/gcsi_ccle/shap_summary.png (deflated 13%)
  adding: Isoform_Exp